# Funky Radical Electrodynamics Engine 1105 #

This Project (which can be abbreviated to FREE1105) Will Exist in 5 Phases:

1) Set Up the Electric Fields in a 2D Coordinate System
2) Simulate the Motion of Electric Point Charges
3) Simluate the Motion of Charged Rigid Bodies
4) Visualise the Field via Geometrical Curvature
5) Implement Maxwell's 4th Law for Magnetic Fields

Bismillah Ar Rahman Ar Rahim - In Sha Allah this will go well!!

## Phase 1 ##

Let us establish the checklist of aims:
- Set Up the window to initialise a screen
- Add a wall on the screen so as to not have particles go so far off screen
- Be able to drag and drop particles on the 2d Screen to form point charges
- Implement these point charges to be able to be static or not
- Implement the necassary equations for superpositions of electric fields
- Visualise the electric field in 2d when static before the simulation

Notes of consideration before starting:
- I want the window and screen to have a constant aspect ratio but be able to be made bigger or smaller
- The wall will have a means of changing the impedence or coefficient of restitution for collisions
- The main difference with particles put in for the electric field vs those put in for test runs of their motion is when the simulation is fully reset, the electric field particles will revert to their original fitr- I mean spots (ba dum tish) whereas the added particles will simply disappear like Monkweh

Alright so In Sha Allah lets do this and vaguely follow some vague classical approximation of physics in the process!!

In [ ]:
import numpy as np
import pygame
import sys

# the set up

pygame.init() # initialises pygame
sw = 1500 # screen width
sh = 900 # screen height
bgc = (200,200,200) # background colour
wc = (0,0,0) # wall colour
bt = 20 # wall border thickness
bw_coeff = 1 # border wall elasticity coefficient

screen = pygame.display.set_mode((sw,sh)) # shows up the screen
pygame.display.set_caption("FREE: Phase 1") # sets a caption for the window
clock = pygame.time.Clock() # clock to cap fps

# the components go here

class dropdown_menu:

    options_list = ["Point Charges"]

    def __init__(self, position, width, height_per_option, options_list, selected = 0):
        self.position = position
        self.width = width
        self.height_per_option = height_per_option
        self.options_list = options_list
        self.selected = selected
        self.expanded = True
        self.font = pygame.font.SysFont('Console', 14)

    def render_menu(self, screen):
        if self.expanded:
            for i in range(len(self.options_list)):
                pygame.draw.rect(screen, (255,255,255), (self.position[0], self.position[1] + i*self.height_per_option, self.width, self.height_per_option))
                pygame.draw.rect(screen, (0,0,0), (self.position[0], self.position[1] + i*self.height_per_option, self.width, self.height_per_option), 2)
                option_text = self.font.render(self.options_list[i], True, (0,0,0))
                screen.blit(option_text, (self.position[0] + 5, self.position[1] + i*self.height_per_option + 5))
        else:
            pygame.draw.rect(screen, (255,255,255), (self.position[0], self.position[1], self.width, self.height_per_option))
            pygame.draw.rect(screen, (0,0,0), (self.position[0], self.position[1], self.width, self.height_per_option), 2)
            option_text = self.font.render(self.options_list[self.selected], True, (0,0,0))
            screen.blit(option_text, (self.position[0] + 5, self.position[1] + 5))

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pos = pygame.mouse.get_pos()
            if self.expanded:
                for i in range(len(self.options_list)):
                    option_rect = pygame.Rect(self.position[0], self.position[1] + i*self.height_per_option, self.width, self.height_per_option)
                    if option_rect.collidepoint(mouse_pos):
                        self.selected = i
                        self.expanded = False
            else:
                main_rect = pygame.Rect(self.position[0], self.position[1], self.width, self.height_per_option)
                if main_rect.collidepoint(mouse_pos):
                    self.expanded = True

dropdown_menu_rc = dropdown_menu(np.array([0,0]), 100, 25, dropdown_menu.options_list)

class PointCharge:

    e_charge = 1.602e-19 # elementary charge in coulombs
    e_mass = 9.109e-31 # electron mass in kg

    def __init__(self, charge, mass, position, velocity, environmental, static):
    
        self.charge = charge * PointCharge.e_charge
        self.mass = mass * PointCharge.e_mass
        self.position = np.array([position[0], position[1]]) 
        self.velocity = np.array(velocity[0], velocity[1])
        if self.charge > 0:
            self.colour = (255,0,0)
        elif self.charge < 0:
            self.colour = (0,0,255)
        else:
            self.colour = (0,0,0)
        self.radius = 30
        self.border = self.radius + 6 
        self.environmental = environmental
        self.static = static

    def get_radius_from_charge(point_charge, all_point_charges=np.array([])): # calculates radius based on relative charge magnitude

        weakest_charge_magnitude = 100**100
        strongest_charge_magnitude = 0

        for pc in all_point_charges:
            charge_magnitude = abs(pc.charge)
            if charge_magnitude < weakest_charge_magnitude:
                weakest_charge_magnitude = charge_magnitude
            if charge_magnitude > strongest_charge_magnitude:
                strongest_charge_magnitude = charge_magnitude

        radius = 10 + 40 * (1-(strongest_charge_magnitude - abs(point_charge))/(strongest_charge_magnitude - weakest_charge_magnitude))
        return radius

    def get_border_from_mass(point_charge, all_point_charges=np.array([])): # calculates border thickness based on relative mass magnitude
        
        lightest_mass_magnitude = 100**100
        heaviest_mass_magnitude = 0

        for pc in all_point_charges:
            mass_magnitude = abs(pc.mass)
            if mass_magnitude < lightest_mass_magnitude:
                lightest_mass_magnitude = mass_magnitude
            if mass_magnitude > heaviest_mass_magnitude:
                heaviest_mass_magnitude = mass_magnitude

        border = 2 + 8 * (1-(heaviest_mass_magnitude - abs(point_charge))/(heaviest_mass_magnitude - lightest_mass_magnitude))
        return border

    def determine_pc_size(point_charge, all_point_charges=np.array([])):

        point_charge.radius = PointCharge.get_radius_from_charge(point_charge, all_point_charges)
        point_charge.border = PointCharge.get_border_from_mass(point_charge, all_point_charges)

    def render_pc(self, screen): # draws the black circle first and then the coloured circle on top to create a border effect

        pygame.draw.circle(screen, (0,0,0), (int(self.position[0]), int(self.position[1])), self.border) # draws the border
        pygame.draw.circle(screen, self.colour, (int(self.position[0]), int(self.position[1])), self.radius) # draws the point charge

# the simulation loop where basically EVERYTHING runs

all_point_charges = [] # list to hold all point charges

while True:

    for event in pygame.event.get(): # where inputs are read during run time

        if event.type == pygame.QUIT: # allows you to close the window by clicking the x on the corner
            
            pygame.quit() # uninitialises pygame
            sys.exit() # closes everything innit
        
        if event.type == pygame.MOUSEBUTTONDOWN: # detects mouse clicks

            if event.button == 3: # right click

                mouse_pos = pygame.mouse.get_pos() # gets mouse position
                dropdown_menu_rc.render_menu(screen) # renders the dropdown menu
                dropdown_menu_rc.handle_event(event) # handles the dropdown menu event
                selected_option = dropdown_menu_rc.options_list[dropdown_menu_rc.selected] # gets the selected option
                if selected_option == "Point Charges":

                    # creates a new point charge at the mouse position with default values

                    new_point_charge = PointCharge(charge=1, mass=1, position=mouse_pos, velocity=[0,0], environmental=True, static=True)
                    all_point_charges.append(new_point_charge)
                    # add the new point charge to a list or array of point charges (not shown in this snippet)

    # where the physics, logic and calculations go below to update components

    screen.fill(bgc) # reclears the screen
    pygame.draw.rect(screen, wc, (0,0,sw,bt)) # top wall
    pygame.draw.rect(screen, wc, (0,sh-bt,sw,bt)) # bottom wall
    pygame.draw.rect(screen, wc, (0,0,bt,sh)) # left wall
    pygame.draw.rect(screen, wc, (sw-bt,0,bt,sh)) # right wall

    # where the updated screen is re rendered below with all the components

    for pc in all_point_charges: # assuming all_point_charges is a list or array of PointCharge objects
        PointCharge.determine_pc_size(pc, all_point_charges)
        pc.render_pc(screen)

    pygame.display.flip() # updates the screen
    clock.tick(60) # caps fps to 60 like pc master race

SystemExit: 